### IMPORTS

In [ ]:
import pandas as pd
import os
from pathlib import Path
import sys

### ORGANIZANDO ARQUIVOS

In [ ]:
OUTPUT_PATH = "../data/processed/predictions.xlsx"

BASE_DIR = Path.cwd().parent
sys.path.append(str(BASE_DIR))

from export_predictions_for_powerbi import export_predictions_for_powerbi
from generate_predictions_script import generate_predictions

### RODANDO SCRIPT DE GERAR PREDIÇÕES

In [ ]:
pred_df = generate_predictions()

### CONSTRUÇÃO DE ESTRUTURA 

In [ ]:
#Construir pred_ml apenas para 'producao_total_barris'
VAR = "producao_total_barris"
fin_path = BASE_DIR / "data" / "processed" / "financials_consolidated.xlsx"
hist = pd.read_excel(fin_path)
hist["data"] = pd.to_datetime(hist["data"])

In [ ]:
# histórico de produção (real)
hist_prod = hist[["data", VAR]].rename(columns={VAR: "real"})

In [ ]:
# previsões futuras (saída do generate_predictions)
if "variavel" in pred_df.columns and "previsto" in pred_df.columns:
    future_prod = pred_df[pred_df["variavel"] == VAR][["data", "previsto"]].copy()
    future_prod["data"] = pd.to_datetime(future_prod["data"])
else:
    raise RuntimeError("A saída de generate_predictions() não contém as colunas esperadas ['variavel','previsto'].")


### ORGANIZANDO pred_ml

In [ ]:
pred_ml = pd.merge(
    hist_prod,
    future_prod.rename(columns={"previsto": "rf_previsto"}),
    on="data",
    how="outer"
).sort_values("data").reset_index(drop=True)

pred_ml["xgb_previsto"] = pred_ml["rf_previsto"] #duplicando a previsão em xgb_previsto


In [ ]:
# preencher com NaN (se for substituir por outro modelo, rode e preencha aqui) MAS o XGB foi o melhor no treino anterior.
pred_ml["xgb_previsto"] = pd.NA

In [ ]:
# reorganizar colunas para o formato exato requerido
pred_ml = pred_ml[["data", "real", "rf_previsto", "xgb_previsto"]]

### CARREGANDO METRICS_DF

In [ ]:
#Carregar metrics_df 
metrics_path = BASE_DIR / "models" / "all_metrics.csv"
if metrics_path.exists():
    metrics_df = pd.read_csv(metrics_path)
    display(metrics_df)
else:
    print("metrics_df não encontrado em:", metrics_path)
    metrics_df = pd.DataFrame()

### RODANDO SCRIPT PARA GERAR PREDIÇÕES PARA POWERBI

In [ ]:
#Chamar export (gera data/processed/predictions.xlsx)
out = export_predictions_for_powerbi(
    pred_ml=pred_ml,
    metrics_df=metrics_df,
    pred_sarima=None,
    company_name="Petroleira Gamarra",
    output_path=str(BASE_DIR / "data" / "processed" / "predictions.xlsx")
)

### INSPEÇÃO PRA VER SE DEU TUDO CERTO

In [ ]:
#Ler o Excel gerado e mostrar abas para inspeção
out_path = BASE_DIR / "data" / "processed" / "predictions.xlsx"
print("Arquivo salvo")
if out_path.exists():
    xls = pd.ExcelFile(out_path)
    print("Abas encontradas:", xls.sheet_names)
    for sheet in xls.sheet_names:
        print(f"\n--- Aba: {sheet} ---")
        df_sheet = pd.read_excel(out_path, sheet_name=sheet)
        display(df_sheet.head())
else:
    print("Erro: arquivo não encontrado após export.")